# Raw обработка собраных titles

In [250]:
import pandas as pd
import glob
import os
from bs4 import BeautifulSoup
import seaborn as sns

In [3]:
#собираю названия всех папок по которым разложены данные
path = r'./urls/output' 
all_holders = glob.glob(os.path.join(path,'*'))
    

In [4]:
#собираю названия всех путей к файлам по которым разложены данные (с названиями файлов)
all_files=[]
for i in all_holders:
    all_files.append(glob.glob(os.path.join(i,'*')))

In [6]:
#собираю все ulr и пути
urls=[]
roots_unfolded=[]
for i in range(len(all_holders)):
    names=[]
    for j in all_files[i]:
        names.append(j.replace((all_holders[i]+"/"),""))
        roots_unfolded.append(j)
    urls.extend(names)
        
        

In [7]:
%%time
titles=[]
err=[]
for i in range(len(roots_unfolded)):
    try:
        titles.append(BeautifulSoup(open(roots_unfolded[i])).title.string)
        #print(BeautifulSoup(open(roots_unfolded[i])).title.string)
    except:
        err.append(i)
        titles.append(urls[i])
       # print(urls[i])
    

#cобрали все тайтлы, а где была ошибка - прометили error


/home/admin/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


CPU times: user 1h 27min 23s, sys: 6min 55s, total: 1h 34min 19s
Wall time: 1h 58min 25s


In [56]:
#Собираем url и тайтлы в один датасет
df = pd.DataFrame(zip(urls,titles))
df=df.reset_index()
df['is_parced'] = df['index'].apply(lambda x: ~(x in err))
df.columns = ['index','url','title','is_parced']

Там, где отсутсвует тайтл - вместо него указываем адрес + делаем в столбце is_parced соотвествующую отметку

In [70]:
df.loc[df['title'].isna(),'is_parced'] = -2
df.loc[df['title'].isna(),'title'] = df.loc[df['title'].isna(),'url']

Создаю вспомагательную таблицу, благодаря которой я могу проанализировать пвторяющиеся заголовки на предмет их адекватности. из самых популярных заголовков вручную промечаю неинформативные ("главная" "страница", "заголовок" и пр). Отфильтровываю их

In [89]:
help_pivot = df.pivot_table(index ='title',values='index',aggfunc='count').sort_values(by='index',ascending=False)
help_pivot = help_pivot.reset_index()
help_pivot.columns=['title','title_count']
help_pivot=help_pivot.loc[help_pivot['title_count']>1]

In [173]:
f = []
for i in range(1,13),range(14,25),range(27,29), range(30,32),range(36,42),[43,45,46,48,49,50,51,52],range(54,60),range(62,66),[67,68,70],range(73,75),range(76,80),range(82,84), range(85,94), range(100,104),range(105,107), [109],range(111,115),range(116,119),range(121,128),range(129,136),[155,158,160],range(161,166),[167,173,177,185,186,187,190,192,193,196,198,204,208,209,213,214,218,220,221,224,229,232,233,235,237,239,240,241,249,252,254,255,256,259,260]:
    for j in i:       
        f.append(j)

In [179]:
df.loc[df['title'].isin(help_pivot.loc[f,'title']),'is_parced']=-2

In [181]:
df.loc[df['title'].isin(help_pivot.loc[f,'title']),'title']=\
df.loc[df['title'].isin(help_pivot.loc[f,'title']),'url']

In [215]:
#загружаю таблицу со всеми url и кол-вом ползоватлей, заходивших на них
full_url = pd.read_csv('urls.csv')
full_url.columns = ['url','us_count']

Затем добавляю отсусвующие url и формирую таблицу для дальнейшей работы с тайтлами и кластеризацией

In [216]:
df2=df.copy()
full_url=full_url.set_index('url')
df2=df2.set_index('url')
result = full_url.join(df2)
result=result.reset_index()
result = result[['url','us_count','title','is_parced']]

In [217]:
result.loc[result['title'].isna(),'is_parced'] = -2
result.loc[result['title'].isna(),'title'] = result.loc[result['title'].isna(),'url']

In [225]:
result=result.dropna()

In [275]:
result.to_csv('titles_sib')

## Следующий этап работы см. файл : mts_ml_cup_sibrikova_main_body.ipynb